Model Training for House Price Prediction of USA
-

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install xgboost
!pip install lightgbm
!pip install catboost


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import os


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# filepath = '/content/drive/My Drive/Colab Notebooks/'

Load file
-

In [30]:
df=pd.read_csv("clean_houseprice_data_1.csv")
df.head()

,livingArea in sq.ft,beds,baths,address,zipcode,city,state,country,price in $
0,3563.491860,8,7,"7658 W Pocahontas Ave, Tampa, FL 33615, Anchor...",99516,Anchorage,AK,USA,1.003360e+06
1,560.000000,1,1,"8057 Reynolds St, Staten Island, NY 10305, Ken...",99611,Kenai,AK,USA,1.404153e+05
2,2390.959601,5,5,"6796 W Comanche Ave, Tampa, FL 33603, Nenana, ...",99760,Nenana,AK,USA,5.281063e+05
3,560.000000,1,1,"1159 Saleta St, Pensacola, FL 32534, North Pol...",99705,North Pole,AK,USA,1.454507e+05
4,560.000000,1,1,"6721 Prospect St NW, Washington, DC 20007, Pal...",99645,Palmer,AK,USA,1.433320e+05


In [31]:
df['price in $']=df['price in $'].astype('float32')
df['price in $']=df['price in $'].astype('int64')
df.head()

,livingArea in sq.ft,beds,baths,address,zipcode,city,state,country,price in $
0,3563.491860,8,7,"7658 W Pocahontas Ave, Tampa, FL 33615, Anchor...",99516,Anchorage,AK,USA,1003360
1,560.000000,1,1,"8057 Reynolds St, Staten Island, NY 10305, Ken...",99611,Kenai,AK,USA,140415
2,2390.959601,5,5,"6796 W Comanche Ave, Tampa, FL 33603, Nenana, ...",99760,Nenana,AK,USA,528106
3,560.000000,1,1,"1159 Saleta St, Pensacola, FL 32534, North Pol...",99705,North Pole,AK,USA,145450
4,560.000000,1,1,"6721 Prospect St NW, Washington, DC 20007, Pal...",99645,Palmer,AK,USA,143332


In [32]:
df = df.drop(columns=['country'])
df.columns

Index(['livingArea in sq.ft', 'beds', 'baths', 'address', 'zipcode', 'city',
       'state', 'price in $'],
      dtype='object')

In [33]:
# One-hot encode state, city, and country
df_encoded = pd.get_dummies(df, columns=['state', 'city'])


Encoders
-

In [34]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded = encoder.fit_transform(df[['state', 'city']])

# Convert back to DataFrame
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['state','city']))

# Concatenate with original DataFrame
df_encoded = pd.concat([df.drop(columns=['state','city']), encoded_df], axis=1)
df_encoded

,livingArea in sq.ft,beds,baths,address,zipcode,price in $,state_AL,state_AZ,state_CA,state_DC,...,city_Windermere,city_Wolverine Lake,city_Woodhaven,city_Woodstock,city_Wrangell,city_Wyandotte,city_Wylie,city_Yorba Linda,city_Ypsilanti,city_Yuma
0,3563.491860,8,7,"7658 W Pocahontas Ave, Tampa, FL 33615, Anchor...",99516,1003360,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,560.000000,1,1,"8057 Reynolds St, Staten Island, NY 10305, Ken...",99611,140415,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2390.959601,5,5,"6796 W Comanche Ave, Tampa, FL 33603, Nenana, ...",99760,528106,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,560.000000,1,1,"1159 Saleta St, Pensacola, FL 32534, North Pol...",99705,145450,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,560.000000,1,1,"6721 Prospect St NW, Washington, DC 20007, Pal...",99645,143332,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,1822.764158,4,4,"8691 Eaton St, Key West, FL 33040, Henrico, VA...",23229,590319,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179996,1597.964758,3,3,"2645 Chadwick Dr, Tampa, FL 33635, Richmond, V...",23222,314642,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179997,4445.693591,10,10,"4820 Center St, Henrico, VA 23229",23229,1250967,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179998,5586.193955,10,9,"5077 Nuthatch Ridge, New Braunfels, TX 78130, ...",23233,1604445,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df=df_encoded
x1 = df.iloc[:, :3]
x2 = df.iloc[:, 4]
x3 = df.iloc[:, 6:]
x= pd.concat([x1,x3], axis=1)
y=df.iloc[ :, 5].values


In [36]:
x

,livingArea in sq.ft,beds,baths,state_AL,state_AZ,state_CA,state_DC,state_FL,state_GA,state_IN,...,city_Windermere,city_Wolverine Lake,city_Woodhaven,city_Woodstock,city_Wrangell,city_Wyandotte,city_Wylie,city_Yorba Linda,city_Ypsilanti,city_Yuma
0,3563.491860,8,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,560.000000,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2390.959601,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,560.000000,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,560.000000,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,1822.764158,4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179996,1597.964758,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179997,4445.693591,10,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179998,5586.193955,10,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
y

array([1003360,  140415,  528106, ..., 1250967, 1604445,  271580],
      shape=(180000,))

Train test split
-

In [38]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

Scaling
-

In [39]:
from sklearn.preprocessing import StandardScaler

# Only scale livingArea
num_cols = ['livingArea in sq.ft']

scaler = StandardScaler().set_output(transform='pandas')

x_train = x_train.copy()
x_test = x_test.copy()

x_train[num_cols] = scaler.fit_transform(x_train[num_cols])
x_test[num_cols] = scaler.transform(x_test[num_cols])


Linear Regression

In [40]:
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(x_train,y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [41]:
y_pred2=regressor.predict(x_test)

In [42]:
mse = mean_squared_error(y_test, y_pred2)
mae = mean_absolute_error(y_test, y_pred2)
r2 = r2_score(y_test, y_pred2)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R² Score:", r2)

Mean Squared Error: 397666462012.173
Mean Absolute Error: 311935.57821104105
R² Score: 0.6788359851137843


XGBoost Model with hyperparameter tuning
-

In [ ]:
# XGBoost
trees = [50, 100, 150, 200]
rates = [0.01, 0.1, 0.2, 0.5, 0.8]
depths = [None, 2, 3, 4, 5]

scores = []   # initialize list only once

for t in tqdm(trees):
    for d in depths:
        for r in rates:
            x_gbm = XGBRegressor(random_state=42, max_depth=d, learning_rate=r, n_estimators=t)
            x_gbm.fit(x_train, y_train)
            y_pred = x_gbm.predict(x_test)
            scores.append(['XGBoost', f'n={t}, rate={r}, depth={d}', r2_score(y_test, y_pred)])

100%|██████████| 4/4 [02:24<00:00, 36.06s/it]


Decision Tree hyperparameter Tuning
-

In [44]:
#  Decision Tree Tuning

depths =[None,3,4,5,6,7]
min_samples = [2,10,0.025,0.01,0.05,0.1]
min_leaf = [1,10,0.025,0.01,0.05,0.1]
for d in tqdm(depths):
    for s in min_samples:
        for f in min_leaf:
            dtc = DecisionTreeRegressor(random_state=42,
                                         max_depth=d,min_samples_split=s,min_samples_leaf=f)
            dtc.fit(x_train,y_train)
            y_pred = dtc.predict(x_test)
            scores.append(['Decision Tree', f'depth={d}, split={s},leaves={f}',r2_score(y_test, y_pred)])

100%|██████████| 6/6 [07:13<00:00, 72.26s/it]


Random Forest hyperparameter Tuning
-

In [45]:
#  Random Forest Tuning

features =[None, 'sqrt', 'log2']
n_est = [25, 50,100,150, 200]
for f in tqdm(features):
    for n in tqdm(n_est):
        rf = RandomForestRegressor(random_state=42, max_features=f, n_estimators=n, 
                                   n_jobs=-1   # parallelize across all cores
)
        rf.fit(x_train, y_train)
        y_pred = rf.predict(x_test)
        scores.append(['Random Forest', f'feat={f}, n={n}', r2_score(y_test, y_pred)])

100%|██████████| 3/3 [04:29<00:00, 89.95s/it] 


Light GBM  model with hyperparameter tuning

In [ ]:
from lightgbm import LGBMRegressor
trees= [50, 100, 150, 200]
rates = [0.01, 0.1, 0.2, 0.001]
depths=[2, 3, 4, 5,7]

for t in tqdm(trees):
    for d in depths:
        for r in rates:
            l_gbm=LGBMRegressor(random_state=42,verbose=-1, max_depth=d, learning_rate=r,n_estimators=t)
            l_gbm.fit(x_train, y_train)
            y_pred= l_gbm.predict(x_test)
            scores.append(['LightGBM', f'n={t}, rate={r}, depth={d}', r2_score(y_test, y_pred)])

100%|██████████| 4/4 [00:49<00:00, 12.33s/it]


Cat Gradient Boosting Regressor

In [ ]:
from catboost import CatBoostRegressor

trees= [50, 100, 150, 200]
rates = [0.01, 0.1, 0.2, 0.001]
depths=[2, 3, 4, 5,7]

for t in tqdm(trees):
    for d in depths:
        for r in rates:
            c_gbm=CatBoostRegressor(random_state=25,verbose=False, max_depth=d, learning_rate=r,n_estimators=t)
            c_gbm.fit(x_train, y_train)
            y_pred= c_gbm.predict(x_test)
            scores.append(['CatBoost', f'n={t}, rate={r}, depth={d}', r2_score(y_test, y_pred)])

100%|██████████| 4/4 [01:26<00:00, 21.71s/it]


Elastic Net
-

In [48]:
from sklearn.linear_model import ElasticNet

# Define parameter grids
alphas = [0.01, 0.1, 1, 10, 100]          # regularization strength
l1_ratios = [0.2, 0.5, 0.8]               # mix of L1 (Lasso) and L2 (Ridge)

# Loop through combinations
for a in tqdm(alphas, desc="Alpha values"):
    for l1 in tqdm(l1_ratios, desc=f"L1 ratio for alpha={a}", leave=False):
        enet = ElasticNet(alpha=a, l1_ratio=l1, random_state=42, max_iter=10000)
        enet.fit(x_train, y_train)
        y_pred = enet.predict(x_test)
        scores.append(['ElasticNet',f'alpha={a}, l1_ratio={l1}',r2_score(y_test, y_pred)])

Alpha values: 100%|██████████| 5/5 [37:12<00:00, 446.52s/it]


In [49]:
# After all loops Convert to DataFrame
score_df = pd.DataFrame(scores, columns=['Model', 'Parameters', 'R2'])
score_df = score_df.sort_values(by='R2', ascending=False) # sort by R² (higher is better)

# Get best model info
best_model_info = score_df.iloc[0]
print("Best Model:", best_model_info['Model'])
print("Parameters:", best_model_info['Parameters'])
print("R²:", best_model_info['R2'])

Best Model: Random Forest
Parameters: feat=None, n=200
R²: 0.9938045280428047


In [50]:
score_df

,Model,Parameters,R2
320,Random Forest,"feat=None, n=200",0.993805
317,Random Forest,"feat=None, n=50",0.993722
319,Random Forest,"feat=None, n=150",0.993709
318,Random Forest,"feat=None, n=100",0.993532
316,Random Forest,"feat=None, n=25",0.993375
...,...,...,...
426,CatBoost,"n=50, rate=0.001, depth=5",0.074968
422,CatBoost,"n=50, rate=0.001, depth=4",0.073143
418,CatBoost,"n=50, rate=0.001, depth=3",0.068951
334,LightGBM,"n=50, rate=0.001, depth=2",0.066723


Best Model
-

In [51]:
# Re-train best Random Forest model
best_model = RandomForestRegressor(
    random_state=42,
    max_features=None,      # use all features
    n_estimators=200,       # best number of trees
    n_jobs=-1               # parallelize for speed
)

best_model.fit(x_train, y_train)

# Training performance
y_train_pred = best_model.predict(x_train)
r2_train = r2_score(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)

# Test performance
y_test_pred = best_model.predict(x_test)
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)

# Print results
print("Train R²:", r2_train)
print("Test R²:", r2_test)
print("Train RMSE:", rmse_train)
print("Test RMSE:", rmse_test)
print("Train MSE:", mse_train)
print("Test MSE:", mse_test)
print("Train MAE:", mae_train)
print("Test MAE:", mae_test)

Train R²: 0.9988712177768787
Test R²: 0.9938045280428047
Train RMSE: 38177.96546401561
Test RMSE: 87585.70726361148
Train MSE: 1457557046.9715683
Test MSE: 7671256116.867044
Train MAE: 4728.9390675
Test MAE: 12016.602320972223


1. R² = 0.9938
- Model explains 99.4% of the variance in house prices.
- This is an excellent fit, especially given the huge spread of values.
2. MAE = $12,016
- On average, predictions are off by about $12k
- Relative to the range:
- For a $100k home -> ~12% error.
- For a $1M home -> ~1.2% error.
- For a $29M home -> ~0.04% error.
3. RMSE = $87,585
- Typical error magnitude.
4. MSE = 7.67 billion
- This is just the squared error scale — not directly interpretable in dollars

------------------------------------------------------------
- Model is excellent overall — very high R² and low average error.
- Absolute Errors are tiny for luxury homes but relatively large for cheaper homes.

Average percentage error

In [52]:
mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100 
print("MAPE:", mape, "%")

MAPE: 0.9712698770707575 %


Cross Validation
-

In [53]:
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Define K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate using R²
cv_scores = cross_val_score(best_model, x, y, cv=kf, scoring='r2')

print("Cross-validation R² scores:", cv_scores)
print("Mean R²:", np.mean(cv_scores))
print("Std deviation:", np.std(cv_scores))


Cross-validation R² scores: [0.99394945 0.99179455 0.99180639 0.99097152 0.99169987]
Mean R²: 0.9920443546506659
Std deviation: 0.001001807497948167


End
-